In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Dec 27 10:14:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Notes for MU-Ra-US = ctc681 SW

- Provide a requirements file with the correct list of dependencies:

> !pip install segmentation-models-pytorch (missing)

> !pip install connected-components-3d

- I need to install imagecodecs to process the images of the CTC, but this can vary according to the data we want to use. 

- Bash code is looking for a specific data folder that is not reported in the quidelines. It is recommendable to explain where and under which names these should be located. `FileNotFoundError: [Errno 2] No such file or directory: '../BF-C2DL-HSC/01_RES-GT\r'`

- The new model that I got has the name `st_sw_DIC-C2DH-HeLa_best_model` but the test code is looking for `gt_st_sw_{args.dataset}_best_model.pth`.

- For the training: is it necessary to have the silver truth? Could it be possible to load a pretrained model?

- There are different train scripts (.sh and .py, with _all, _gt_all). What is each of them for?

- Is the training taking all the sequences (01 and 02) of the dataset? If I had my own data, could I have folders named as 01, 02, 03,.. and run the training for them?

In [ ]:
import os

os.chdir("/content/drive/MyDrive/Estibaliz Gomez de Mariscal/CELL TRACKING CHALLENGE (CTC)/CTC trainable solutions/Data Estibaliz/MU-Ra-US = ctc681 SW/U-SE-ResNet-for-Cell-Tracking-Challenge/SW")
!pip install -r requirements.txt

     |████████████████████████████████| 104 kB 6.3 MB/s 
     |████████████████████████████████| 150 kB 17.3 MB/s 
     |████████████████████████████████| 156 kB 28.4 MB/s 
     |████████████████████████████████| 1.9 MB 39.1 MB/s 
     |████████████████████████████████| 3.7 MB 60.6 MB/s 
     |████████████████████████████████| 2.0 MB 50.1 MB/s 
     |████████████████████████████████| 81 kB 12.8 MB/s 
     |████████████████████████████████| 829 kB 48.6 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 159 kB 75.3 MB/s 
     |████████████████████████████████| 76 kB 7.0 MB/s 
     |████████████████████████████████| 57 kB 6.7 MB/s 
     |████████████████████████████████| 2.7 MB 52.2 MB/s 
     |████████████████████████████████| 2.9 MB 53.6 MB/s 
     |████████████████████████████████| 49 kB 6.8 MB/s 
     |████████████████████████████████| 58 kB 8.2 MB/s 
     |████████████████████████████████| 3.3 MB 55.7 MB/s 
     |████████████████████

In [ ]:
!pip install segmentation-models-pytorch

     |████████████████████████████████| 88 kB 4.1 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
     |████████████████████████████████| 376 kB 14.0 MB/s 
  Using cached efficientnet_pytorch-0.6.3.tar.gz (16 kB)
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=9654595b46b3b798393f311b6e345179510964e51aa7bf129f7e61f56085302c
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=f99d11d190dfc86b5a32b6230b56ef3f8501f84a1072ac88f4b3e4d048e3b4ad
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels


In [ ]:
!pip install connected-components-3d

     |████████████████████████████████| 2.2 MB 5.3 MB/s 


In [ ]:
!pip install imagecodecs

     |████████████████████████████████| 31.0 MB 1.3 MB/s 


# Training with DIC-C2DH-HeLa dataset

In [ ]:
## Download the training data from the CTC in the Data/train folder of the software
os.chdir("/content/drive/MyDrive/Estibaliz Gomez de Mariscal/CELL TRACKING CHALLENGE (CTC)/CTC trainable solutions/Data Estibaliz/MU-Ra-US = ctc681 SW/U-SE-ResNet-for-Cell-Tracking-Challenge/Data/train")
!wget --no-check-certificate http://data.celltrackingchallenge.net/training-datasets/DIC-C2DH-HeLa.zip
!unzip DIC-C2DH-HeLa.zip
!rm -r DIC-C2DH-HeLa.zip

--2021-12-14 14:16:57--  http://data.celltrackingchallenge.net/training-datasets/DIC-C2DH-HeLa.zip
Resolving data.celltrackingchallenge.net (data.celltrackingchallenge.net)... 147.251.52.183
Connecting to data.celltrackingchallenge.net (data.celltrackingchallenge.net)|147.251.52.183|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://data.celltrackingchallenge.net/training-datasets/DIC-C2DH-HeLa.zip [following]
--2021-12-14 14:16:57--  https://data.celltrackingchallenge.net/training-datasets/DIC-C2DH-HeLa.zip
Connecting to data.celltrackingchallenge.net (data.celltrackingchallenge.net)|147.251.52.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38569954 (37M) [application/zip]
Saving to: ‘DIC-C2DH-HeLa.zip’

DIC-C2DH-HeLa.zip   100%[===================>]  36.78M  42.0MB/s    in 0.9s    

2021-12-14 14:16:58 (42.0 MB/s) - ‘DIC-C2DH-HeLa.zip’ saved [38569954/38569954]

Archive:  DIC-C2DH-HeLa.zip
replace DIC-C2DH-

In [ ]:
# Run the training script
os.chdir("/content/drive/MyDrive/Estibaliz Gomez de Mariscal/CELL TRACKING CHALLENGE (CTC)/CTC trainable solutions/Data Estibaliz/MU-Ra-US = ctc681 SW/U-SE-ResNet-for-Cell-Tracking-Challenge/SW/train_codes")
!python train.py --dataset DIC-C2DH-HeLa --erosion 2

False
Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnet50-ce0d4300.pth" to /root/.cache/torch/hub/checkpoints/se_resnet50-ce0d4300.pth
100% 107M/107M [05:16<00:00, 355kB/s]
Loading data...
100% 151/151 [00:00<00:00, 154.64it/s]
100% 151/151 [00:02<00:00, 52.99it/s]
100% 151/151 [00:15<00:00,  9.89it/s]
100% 17/17 [00:00<00:00, 164.05it/s]
100% 17/17 [00:00<00:00, 52.85it/s]
100% 17/17 [00:01<00:00,  9.40it/s]
Preparing for training...
Training...
 24% 29/120 [08:52<25:50, 17.04s/it]Decrease decoder learning rate to 5e-05
 49% 59/120 [17:29<17:13, 16.94s/it]Decrease decoder learning rate to 2.5e-05
 74% 89/120 [25:58<08:34, 16.59s/it]Decrease decoder learning rate to 1.25e-05
 99% 119/120 [34:22<00:16, 16.84s/it]Decrease decoder learning rate to 6.25e-06
100% 120/120 [34:39<00:00, 17.33s/it]


# Inference with DIC-C2DH-HeLa dataset

In [ ]:
# Run the inference
os.chdir("/content/drive/MyDrive/Estibaliz Gomez de Mariscal/CELL TRACKING CHALLENGE (CTC)/CTC trainable solutions/Data Estibaliz/MU-Ra-US = ctc681 SW/U-SE-ResNet-for-Cell-Tracking-Challenge/SW/test_codes")
# Set execute permission
!chmod +x DIC-C2DH-HeLa-01.sh
!./DIC-C2DH-HeLa-01.sh

Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask000.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask001.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask002.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask003.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask004.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask005.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask006.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask007.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask008.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask009.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask010.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask011.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask012.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask013.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa/01_RES/mask014.tif ...
Saving to: ../../Data/test/DIC-C2DH-HeLa